## Importy

In [3]:
import torch, torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import torch.utils.data as data
import numpy as np
import pandas as pd
import matplotlib
import random
import seaborn as sns

## Ustawienia

In [4]:
random.seed(42)
pd.set_option('display.max_columns', None)   # wyświetlanie wszystkich kolumn


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

## Analiza dostarczonych danych

Jak wyglądają dane?

In [5]:
raw_data = pd.read_csv('train_data.csv')
raw_data.head()

,SalePrice,YearBuilt,YrSold,MonthSold,Size(sqf),Floor,HallwayType,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_APT,N_manager,N_elevators,SubwayStation,N_FacilitiesNearBy(PublicOffice),N_FacilitiesNearBy(Hospital),N_FacilitiesNearBy(Dpartmentstore),N_FacilitiesNearBy(Mall),N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(Park),N_SchoolNearBy(Elementary),N_SchoolNearBy(Middle),N_SchoolNearBy(High),N_SchoolNearBy(University),N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,51327,1985,2007,8,587,8,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,1.0,2.0,2.0,Daegu,5.0,1,2.0,1.0,2.0,1.0,2.0,1.0,1.0,0.0,3,12.0,4.0
1,48672,1985,2007,8,587,6,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,1.0,2.0,2.0,Daegu,5.0,1,2.0,1.0,2.0,1.0,2.0,1.0,1.0,0.0,3,12.0,4.0
2,380530,2006,2007,8,2056,8,terraced,individual_heating,management_in_trust,249.0,536.0,0~5min,0-5min,6.0,5.0,11.0,Sin-nam,1.0,1,0.0,1.0,0.0,0.0,2.0,2.0,1.0,2.0,5,3.0,7.0
3,221238,1993,2007,8,1761,3,mixed,individual_heating,management_in_trust,523.0,536.0,0~5min,15min~20min,8.0,8.0,20.0,Myung-duk,6.0,2,0.0,1.0,5.0,0.0,4.0,3.0,5.0,5.0,4,14.0,17.0
4,35840,1992,2007,8,355,5,corridor,individual_heating,management_in_trust,200.0,0.0,5min~10min,10min~15min,3.0,5.0,10.0,Myung-duk,7.0,1,1.0,1.0,5.0,1.0,4.0,3.0,5.0,5.0,3,16.0,17.0


In [6]:
raw_data.describe()

,SalePrice,YearBuilt,YrSold,MonthSold,Size(sqf),Floor,N_Parkinglot(Ground),N_Parkinglot(Basement),N_APT,N_manager,N_elevators,N_FacilitiesNearBy(PublicOffice),N_FacilitiesNearBy(Hospital),N_FacilitiesNearBy(Dpartmentstore),N_FacilitiesNearBy(Mall),N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(Park),N_SchoolNearBy(Elementary),N_SchoolNearBy(Middle),N_SchoolNearBy(High),N_SchoolNearBy(University),N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
count,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000
mean,221564.653249,2002.907129,2012.684772,6.201261,960.484239,11.994908,195.616392,572.357905,5.607177,6.333172,11.151552,4.141125,1.298982,0.912464,0.947381,1.932590,0.658341,3.018914,2.425800,2.663191,2.753637,5.801164,9.890883,10.861542
std,106676.053778,8.825764,2.917335,3.398540,386.804458,7.548805,218.919683,409.246602,2.818877,3.193502,7.810642,1.775126,0.478587,0.814821,0.396927,2.199886,0.661033,0.940444,1.032846,1.547938,1.491970,2.340055,3.403815,4.417403
min,34070.000000,1978.000000,2007.000000,1.000000,135.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,144247.000000,1993.000000,2010.000000,3.000000,644.000000,6.000000,11.000000,184.000000,3.000000,5.000000,5.000000,3.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,4.000000,8.000000,7.000000
50%,207964.000000,2006.000000,2013.000000,6.000000,910.000000,11.000000,100.000000,536.000000,7.000000,6.000000,11.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,3.000000,2.000000,2.000000,5.000000,9.000000,10.000000
75%,289380.000000,2007.000000,2015.000000,9.000000,1160.000000,17.000000,249.000000,798.000000,8.000000,8.000000,16.000000,5.000000,2.000000,2.000000,1.000000,5.000000,1.000000,4.000000,3.000000,4.000000,4.000000,7.000000,13.000000,15.000000
max,585840.000000,2015.000000,2017.000000,12.000000,2337.000000,43.000000,713.000000,1321.000000,13.000000,14.000000,27.000000,7.000000,2.000000,2.000000,2.000000,5.000000,2.000000,6.000000,4.000000,5.000000,5.000000,10.000000,16.000000,17.000000


In [7]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4124 entries, 0 to 4123
Data columns (total 30 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   SalePrice                           4124 non-null   int64  
 1   YearBuilt                           4124 non-null   int64  
 2   YrSold                              4124 non-null   int64  
 3   MonthSold                           4124 non-null   int64  
 4   Size(sqf)                           4124 non-null   int64  
 5   Floor                               4124 non-null   int64  
 6   HallwayType                         4124 non-null   object 
 7   HeatingType                         4124 non-null   object 
 8   AptManageType                       4124 non-null   object 
 9   N_Parkinglot(Ground)                4124 non-null   float64
 10  N_Parkinglot(Basement)              4124 non-null   float64
 11  TimeToBusStop                       4124 no

Zamiana danych:

In [8]:
categories = ['HallwayType', 'HeatingType', 'AptManageType', 'TimeToBusStop', 'TimeToSubway', 'SubwayStation']
for cat in categories:
    print(raw_data[cat].astype('category').cat.categories)

Index(['corridor', 'mixed', 'terraced'], dtype='object')
Index(['central_heating', 'individual_heating'], dtype='object')
Index(['management_in_trust', 'self_management'], dtype='object')
Index(['0~5min', '10min~15min', '5min~10min'], dtype='object')
Index(['0-5min', '10min~15min', '15min~20min', '5min~10min',
       'no_bus_stop_nearby'],
      dtype='object')
Index(['Bangoge', 'Banwoldang', 'Chil-sung-market', 'Daegu',
       'Kyungbuk_uni_hospital', 'Myung-duk', 'Sin-nam', 'no_subway_nearby'],
      dtype='object')


In [9]:
targ = raw_data['SalePrice']
raw_data = raw_data.drop(['SalePrice'], axis=1)

In [10]:
ht = raw_data["HallwayType"].astype("category")
ht = pd.get_dummies(ht)

In [11]:
substation = raw_data["SubwayStation"].astype("category")
substation = pd.get_dummies(substation)

In [12]:
raw_data['HeatingType'] = raw_data['HeatingType'].replace(['central_heating', 'individual_heating'], [0, 1])

In [13]:
raw_data['AptManageType'] = raw_data['AptManageType'].replace(['management_in_trust', 'self_management'], [0, 1])

In [14]:
raw_data['TimeToBusStop'] = raw_data['TimeToBusStop'].replace(['0~5min', '10min~15min', '5min~10min'], [0, 2, 1])

In [15]:
raw_data['TimeToSubway'] = raw_data['TimeToSubway'].replace(['0-5min', '10min~15min', '15min~20min', '5min~10min',
       'no_bus_stop_nearby'], [1, 3, 4, 2, 0])

In [16]:
raw_data = raw_data.drop(['HallwayType', 'SubwayStation'], axis=1)

normalizacja danych:

In [17]:
means, maxs, mins = dict(), dict(), dict()
mean_targ, max_targ, min_targ = targ.mean(), targ.max(), targ.min()

for col in raw_data:
    means[col] = raw_data[col].mean()
    maxs[col] = raw_data[col].max()
    mins[col] = raw_data[col].min()

In [18]:
raw_data.head()

,YearBuilt,YrSold,MonthSold,Size(sqf),Floor,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_APT,N_manager,N_elevators,N_FacilitiesNearBy(PublicOffice),N_FacilitiesNearBy(Hospital),N_FacilitiesNearBy(Dpartmentstore),N_FacilitiesNearBy(Mall),N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(Park),N_SchoolNearBy(Elementary),N_SchoolNearBy(Middle),N_SchoolNearBy(High),N_SchoolNearBy(University),N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,1985,2007,8,587,8,1,1,80.0,76.0,0,2,1.0,2.0,2.0,5.0,1,2.0,1.0,2.0,1.0,2.0,1.0,1.0,0.0,3,12.0,4.0
1,1985,2007,8,587,6,1,1,80.0,76.0,0,2,1.0,2.0,2.0,5.0,1,2.0,1.0,2.0,1.0,2.0,1.0,1.0,0.0,3,12.0,4.0
2,2006,2007,8,2056,8,1,0,249.0,536.0,0,1,6.0,5.0,11.0,1.0,1,0.0,1.0,0.0,0.0,2.0,2.0,1.0,2.0,5,3.0,7.0
3,1993,2007,8,1761,3,1,0,523.0,536.0,0,4,8.0,8.0,20.0,6.0,2,0.0,1.0,5.0,0.0,4.0,3.0,5.0,5.0,4,14.0,17.0
4,1992,2007,8,355,5,1,0,200.0,0.0,1,3,3.0,5.0,10.0,7.0,1,1.0,1.0,5.0,1.0,4.0,3.0,5.0,5.0,3,16.0,17.0


In [19]:
targ.head()

0     51327
1     48672
2    380530
3    221238
4     35840
Name: SalePrice, dtype: int64

In [20]:
for col in raw_data:
  raw_data[col] = (raw_data[col] - means[col]) / (maxs[col] - mins[col])
raw_data.head()

,YearBuilt,YrSold,MonthSold,Size(sqf),Floor,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_APT,N_manager,N_elevators,N_FacilitiesNearBy(PublicOffice),N_FacilitiesNearBy(Hospital),N_FacilitiesNearBy(Dpartmentstore),N_FacilitiesNearBy(Mall),N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(Park),N_SchoolNearBy(Elementary),N_SchoolNearBy(Middle),N_SchoolNearBy(High),N_SchoolNearBy(University),N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,-0.483976,-0.568477,0.163522,-0.169611,-0.095117,0.050921,0.938167,-0.162155,-0.375744,-0.120635,0.021945,-0.383931,-0.333321,-0.338946,0.122696,-0.149491,0.543768,0.026309,0.013482,0.170829,-0.169819,-0.35645,-0.332638,-0.550727,-0.311240,0.13182,-0.403620
1,-0.483976,-0.568477,0.163522,-0.169611,-0.142736,0.050921,0.938167,-0.162155,-0.375744,-0.120635,0.021945,-0.383931,-0.333321,-0.338946,0.122696,-0.149491,0.543768,0.026309,0.013482,0.170829,-0.169819,-0.35645,-0.332638,-0.550727,-0.311240,0.13182,-0.403620
2,0.083591,-0.568477,0.163522,0.497509,-0.095117,0.050921,-0.061833,0.074872,-0.027523,-0.120635,-0.228055,0.032735,-0.102552,-0.005613,-0.448732,-0.149491,-0.456232,0.026309,-0.386518,-0.329171,-0.169819,-0.10645,-0.332638,-0.150727,-0.089018,-0.43068,-0.227150
3,-0.267760,-0.568477,0.163522,0.363540,-0.214164,0.050921,-0.061833,0.459164,-0.027523,-0.120635,0.521945,0.199402,0.128218,0.327720,0.265554,0.350509,-0.456232,0.026309,0.613482,-0.329171,0.163514,0.14355,0.467362,0.449273,-0.200129,0.25682,0.361086
4,-0.294787,-0.568477,0.163522,-0.274970,-0.166545,0.050921,-0.061833,0.006148,-0.433276,0.379365,0.271945,-0.217265,-0.102552,-0.042650,0.408411,-0.149491,0.043768,0.026309,0.613482,0.170829,0.163514,0.14355,0.467362,0.449273,-0.311240,0.38182,0.361086


In [21]:
targ = (targ - mean_targ) / (max_targ - min_targ)
targ.head()

0   -0.308530
1   -0.313342
2    0.288101
3   -0.000592
4   -0.336598
Name: SalePrice, dtype: float64

podział danych:

In [22]:
train_indicates = np.random.rand(len(raw_data)) > 0.3

In [23]:
numerical_data = torch.from_numpy(raw_data.values[train_indicates]).float()
categorical_ht_data = torch.from_numpy(ht.values[train_indicates]).float()
categorical_substation_data = torch.from_numpy(substation.values[train_indicates]).float()
targets = torch.from_numpy(targ.values[train_indicates]).float()

test_numerical_data = torch.from_numpy(raw_data.values[~train_indicates]).float()
test_categorical_ht_data = torch.from_numpy(ht.values[~train_indicates]).float()
test_categorical_substation_data = torch.from_numpy(substation.values[~train_indicates]).float()
test_targets = torch.from_numpy(targ.values[~train_indicates]).float()

## Budowanie sieci

In [24]:
train_dataset = data.TensorDataset(numerical_data, categorical_ht_data, categorical_substation_data, targets)
test_dataset = data.TensorDataset(test_numerical_data, test_categorical_ht_data, test_categorical_substation_data, test_targets)

In [25]:
numerical_data.shape[1] + categorical_ht_data.shape[1] + categorical_substation_data.shape[1]

38

In [26]:
class Price_classifier(nn.Module):
    def __init__(self):
        super(Price_classifier, self).__init__()
        self.layer1 = nn.Linear(numerical_data.shape[1] + categorical_ht_data.shape[1] + categorical_substation_data.shape[1], 300)
        self.bn1 = nn.BatchNorm1d(300)
        self.act_1 = nn.Sigmoid()
        self.layer2 = nn.Linear(300, 500)
        self.bn2 = nn.BatchNorm1d(500)
        self.act_2 = nn.Sigmoid()
        self.layer5 = nn.Linear(500, 50)
        self.bn5 = nn.BatchNorm1d(50)
        self.act_5 = nn.ReLU()
        self.layer6 = nn.Linear(50, 1)

    def forward(self, x, cat_ht_x, cat_substation_x):
        x = torch.cat([x, cat_ht_x, cat_substation_x], dim=1)
        activation1 = self.act_1(self.bn1(self.layer1(x)))
        activation2 = self.act_2(self.bn2(self.layer2(activation1)))
        activation5 = self.act_5(self.bn5(self.layer5(activation2)))
        output = self.layer6(activation5)
        return output

In [27]:
threshold = 300000

In [47]:
def evaluate(model, data, targets, scale):
  model.eval()
  pred = model(data[0].to(device), data[1].to(device), data[2].to(device))
  pred = pred*(scale['max_val'] - scale['min_val']) + scale['mean_val'] > threshold
  real = targets*(scale['max_val'] - scale['min_val']) + scale['mean_val'] > 300000
  return (pred.squeeze().to(device) == real.squeeze().to(device)).sum().item() / real.shape[0]

In [50]:
scale = {'max_val': max_targ, 'min_val': min_targ, 'mean_val': mean_targ}

In [60]:
model = Price_classifier().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

iters = []
losses = []
train_acc = []
val_acc = []
for n in range(2000):
    epoch_losses = []
    for x, cat_ht_x, cat_substation_x, y in iter(train_loader):
        x, cat_ht_x, cat_substation_x, y = x.to(device), cat_ht_x.to(device), cat_substation_x.to(device), y.to(device)
        model.train()
        out = model(x, cat_ht_x, cat_substation_x).squeeze()

        loss = criterion(out, y)
        loss.backward()
        epoch_losses.append(loss.item())
        optimizer.step()
        optimizer.zero_grad()

    loss_mean = np.array(epoch_losses).mean()
    train_precision = evaluate(model, [numerical_data, categorical_ht_data, categorical_substation_data], targets, scale)
    test_precision = evaluate(model, [test_numerical_data, test_categorical_ht_data, test_categorical_substation_data], test_targets, scale)
    iters.append(n)
    losses.append(loss_mean)
    print(f"Epoch: {n},\t loss: {loss_mean:.4},\t\t train precision: {train_precision:.4},\t\t test precision: {test_precision:.4}")


Epoch: 0,	 loss: 0.01876,		 train precision: 0.9213,		 test precision: 0.9259
Epoch: 1,	 loss: 0.006371,		 train precision: 0.9344,		 test precision: 0.9185
Epoch: 2,	 loss: 0.005317,		 train precision: 0.9326,		 test precision: 0.9308
Epoch: 3,	 loss: 0.004901,		 train precision: 0.91,		 test precision: 0.9209
Epoch: 4,	 loss: 0.004164,		 train precision: 0.9519,		 test precision: 0.9572
Epoch: 5,	 loss: 0.004091,		 train precision: 0.9447,		 test precision: 0.9489
Epoch: 6,	 loss: 0.004,		 train precision: 0.9519,		 test precision: 0.9481
Epoch: 7,	 loss: 0.003826,		 train precision: 0.9505,		 test precision: 0.9498
Epoch: 8,	 loss: 0.003355,		 train precision: 0.9457,		 test precision: 0.9506
Epoch: 9,	 loss: 0.003262,		 train precision: 0.9509,		 test precision: 0.9465
Epoch: 10,	 loss: 0.00318,		 train precision: 0.9584,		 test precision: 0.9596
Epoch: 11,	 loss: 0.002974,		 train precision: 0.9471,		 test precision: 0.944
Epoch: 12,	 loss: 0.003096,		 train precision: 0.9543,		 t

In [65]:
for i in range(280000, 320000, 2000):
  threshold = i
  prec = evaluate(model, [test_numerical_data, test_categorical_ht_data, test_categorical_substation_data], test_targets, scale)
  print(i, ': ', prec)

280000 :  0.9530477759472817
282000 :  0.9555189456342669
284000 :  0.957166392092257
286000 :  0.9596375617792422
288000 :  0.9621087314662273
290000 :  0.9654036243822076
292000 :  0.9686985172981878
294000 :  0.9703459637561779
296000 :  0.9719934102141681
298000 :  0.9761120263591433
300000 :  0.9761120263591433
302000 :  0.9769357495881383
304000 :  0.9752883031301482
306000 :  0.9744645799011532
308000 :  0.9736408566721582
310000 :  0.9719934102141681
312000 :  0.9736408566721582
314000 :  0.9695222405271828
316000 :  0.9670510708401977
318000 :  0.9670510708401977


In [66]:
for i in range(300000, 302000, 200):
  threshold = i
  prec = evaluate(model, [test_numerical_data, test_categorical_ht_data, test_categorical_substation_data], test_targets, scale)
  print(i, ': ', prec)

300000 :  0.9761120263591433
300200 :  0.9761120263591433
300400 :  0.9761120263591433
300600 :  0.9761120263591433
300800 :  0.9761120263591433
301000 :  0.9769357495881383
301200 :  0.9769357495881383
301400 :  0.9769357495881383
301600 :  0.9769357495881383
301800 :  0.9769357495881383


In [83]:
found_threshold = 301000

## Klasyfikacja zbioru testowego

In [79]:
test_raw_data = pd.read_csv('test_data.csv')

test_ht = test_raw_data["HallwayType"].astype("category")
test_ht = pd.get_dummies(test_ht)

test_substation = test_raw_data["SubwayStation"].astype("category")
test_substation = pd.get_dummies(test_substation)

test_raw_data['HeatingType'] = test_raw_data['HeatingType'].replace(['central_heating', 'individual_heating'], [0, 1])
test_raw_data['AptManageType'] = test_raw_data['AptManageType'].replace(['management_in_trust', 'self_management'], [0, 1])
test_raw_data['TimeToBusStop'] = test_raw_data['TimeToBusStop'].replace(['0~5min', '10min~15min', '5min~10min'], [0, 2, 1])
test_raw_data['TimeToSubway'] = test_raw_data['TimeToSubway'].replace(['0-5min', '10min~15min', '15min~20min', '5min~10min',
       'no_bus_stop_nearby'], [1, 3, 4, 2, 0])
test_raw_data = test_raw_data.drop(['HallwayType', 'SubwayStation'], axis=1)

for col in test_raw_data:
  test_raw_data[col] = (test_raw_data[col] - means[col]) / (maxs[col] - mins[col])

test_raw_data = torch.from_numpy(np.array(test_raw_data)).float()
test_ht = torch.from_numpy(np.array(test_ht)).float()
test_substation = torch.from_numpy(np.array(test_substation)).float()

In [91]:
final_predictions = model(test_raw_data.to(device), test_ht.to(device), test_substation.to(device))

In [92]:
final_predictions = final_predictions*(max_targ - min_targ) + mean_targ
final_classes = final_predictions > found_threshold

[tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([True], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([True], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([True], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([True], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device='cuda:0'),
 tensor([False], device=

In [98]:
to_file = zip(list(final_predictions), list(final_classes))
list(to_file)

[(tensor([126196.7500], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([False], device='cuda:0')),
 (tensor([197784.0625], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([False], device='cuda:0')),
 (tensor([146150.9688], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([False], device='cuda:0')),
 (tensor([306252.8125], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([True], device='cuda:0')),
 (tensor([225870.3750], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([False], device='cuda:0')),
 (tensor([255424.4531], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([False], device='cuda:0')),
 (tensor([152897.2812], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([False], device='cuda:0')),
 (tensor([451499.8750], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([True], device='cuda:0')),
 (tensor([193951.1250], device='cuda:0', grad_fn=<UnbindBackward>),
  tensor([False], device='cuda:0')),
 (tensor([210648.2031], device='cuda:0', grad_fn=<UnbindB